In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

nbn = "03_" # Notebook Number; Prefix for artifacts path

In [2]:
datasets = ["DTD","EuroSAT","GTSRB","MNIST","SVHN","Caltech256"]
nrow, ncol=2, 3

In [3]:
def calc_tpr(targ, pred):
    tp = np.sum((targ == 1) & (pred == 1))
    fn = np.sum((targ == 1) & (pred == 0))
    return tp / (tp + fn)
def calc_fpr(targ, pred):
    fp = np.sum((targ == 0) & (pred == 1))
    tn = np.sum((targ == 0) & (pred == 0))
    return fp / (fp + tn)
def predict_correctness(ent_ls,threshold):
    return np.where(ent_ls>threshold,0,1)

In [4]:
def auroc(pred_vec, target_vec, unc_vec, label, ax):
    cor_pred = np.where(pred_vec == target_vec, 1, 0)
    max_unc = unc_vec.max()
    min_unc = unc_vec.min()
    tpr = np.array([]) 
    fpr = np.array([])
    step = (max_unc - min_unc) / 200
    for ent_step in np.arange(min_unc, max_unc+step, step):
        temp = predict_correctness(unc_vec, ent_step)
        tpr = np.append(tpr, calc_tpr(cor_pred, temp))
        fpr = np.append(fpr, calc_fpr(cor_pred, temp))
    area = np.trapz(tpr, fpr)
    sns.lineplot(x=fpr, y=tpr, label=f"{label} (AUC: {area:.2f})", ax=ax)
    return area

In [5]:
pd.read_csv(f"uncertainty_Caltech256.csv", index_col="Unnamed: 0")

kl_18_101  kl_101_18    ent_18   ent_101  ce_18_101  ce_101_18  \
0      0.005454   0.003145  0.066042  0.041488   0.071496   0.044633   
1      2.549316   0.915429  3.886930  0.110614   6.436246   1.026043   
2      0.739175   0.261443  1.947661  0.117020   2.686836   0.378463   
3      2.473712   2.601213  5.076013  2.089456   7.549725   4.690669   
4      0.080690   0.020294  0.222456  0.009736   0.303146   0.030031   
...         ...        ...       ...       ...        ...        ...   
3056   3.374044   2.086314  4.855785  0.267708   8.229829   2.354021   
3057   0.005250   0.005666  0.139103  0.145687   0.144353   0.151353   
3058   1.218661   1.436159  5.206893  3.502298   6.425553   4.938458   
3059   0.031266   0.015188  0.301740  0.099205   0.333006   0.114393   
3060   0.015686   0.022144  0.166051  0.245145   0.181737   0.267290   

        nll_18   nll_101  brier_18     brier_101  
0     0.006201  0.003602  0.000039  1.330926e-05  
1     0.968591  0.010669  0.387984  1.158240e-04  
2     0.305334  0.010830  0.069796  1.167044e-04  
3     2.885286  0.993381  0.899378  5.526911e-01  
4     0.022679  0.000789  0.000507  6.549619e-07  
...        ...       ...       ...           ...  
3056  2.300174  0.038751  0.824796  1.964014e-03  
3057  0.013263  0.013745  0.000175  1.874995e-04  
3058  2.947271  4.732812  0.903805  1.134832e+00  
3059  0.031260  0.009134  0.000953  8.347732e-05  
3060  0.016829  0.025657  0.000284  6.506448e-04  

[3061 rows x 10 columns]

In [6]:
fig, axes = plt.subplots(nrow, ncol, figsize=(25, 15))

for i, ds in enumerate(datasets):
    pred = pd.read_csv(f"prediction_{ds}.csv", index_col="Unnamed: 0")
    unc_pred = pd.read_csv(f"uncertainty_{ds}.csv", index_col="Unnamed: 0")
    
    metrics = [
        (np.log(unc_pred["kl_18_101"]), "kl(18||101)"),
        (np.log(unc_pred["kl_101_18"]), "kl(101||18)"),
        (np.log(unc_pred["ent_18"]), "entropy 18"),
        (np.log(unc_pred["ent_101"]), "entropy 101"),
        (np.log(unc_pred["ce_18_101"]), "cross-entr(18,101)"),
        (np.log(unc_pred["ce_101_18"]), "cross-entr(101,18)"),
        (np.log(unc_pred["ent_101"] * unc_pred["kl_101_18"]), "entr101*kl(101||18)"),
        (np.log(unc_pred["ent_18"] * unc_pred["kl_18_101"]), "entr18*kl(18||101)"),
        (np.log(unc_pred["ent_101"] + unc_pred["ce_101_18"]), "entr101+ce(101||18)"),
        (np.log(unc_pred["ent_101"] * unc_pred["ce_101_18"]), "entr101*ce(101||18)")
    ]
    
    ax = axes[i // ncol, i % ncol]
    for metric, label in metrics:
        ax.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line
        auroc(pred["pred101"], pred["target"], metric, label, ax)
        ax.set_xlabel("False Positive Rate")
        ax.set_ylabel("True Positive Rate")
        ax.set_title(f"AUCROC for Resnet101's Correctness on {ds}")
        ax.legend(loc="lower right")

plt.suptitle("AUCROC for Resnet101's Correctness across Datasets")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the main title
plt.savefig(f"artifacts/{nbn}aucroc.png")
plt.show()

/Users/timothyzhou/anaconda3/envs/ml/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/timothyzhou/anaconda3/envs/ml/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/timothyzhou/anaconda3/envs/ml/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/timothyzhou/anaconda3/envs/ml/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/timothyzhou/anaconda3/envs/ml/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
